In [1]:
import pickle

In [2]:
from spectroscopy.data import (
    parse_trm_files,
    parse_lab_reports,
    extract_data,
    get_unmatched_sample_ids,
    SAMPLE_IDENTIFIER_COLUMNS,
    LAB_REPORT_COLUMNS
)

In [3]:
from spectroscopy.app_utils import get_training_data_path

In [4]:
from spectroscopy.utils import get_wavelength_columns

In [5]:
data_path = get_training_data_path()

no configuration file found at config.ini


In [6]:
skip_paths = None

In [7]:
concurrent = True

In [8]:
df_trms, trm_filepaths = parse_trm_files(
    data_dir=data_path,
    skip_paths=skip_paths,
    concurrent=concurrent
)

In [9]:
len(trm_filepaths)

484

# make sure that trm files were parsed correctly`

### drop the wavelength columns

In [10]:
df_trms.drop(columns=get_wavelength_columns(df_trms), inplace=True)

In [11]:
df_trms.head()

,extra_info,integration_time,filename,sample_name,sample_date,run_number,process_method
0,File: Users\stell\Desktop\StellarDat\North 15...,40,North 15 - 8-13-20 #1.TRM,north 15,2020-08-13,1,
0,File: Users\stell\Desktop\StellarDat\South 10...,40,South 10 - 7-8-20 #2.TRM,south 10,2020-07-08,2,
0,File: Users\stell\Desktop\StellarDat\West 3 -...,30,West 3 - 7-6-20 #3.TRM,west 3,2020-07-06,3,
0,File: Users\stell\Desktop\StellarDat\West 14 ...,30,West 14 - 8-13-20 #1.TRM,west 14,2020-08-13,1,
0,File: Users\stell\Desktop\StellarDat\Dows Wet...,35,Dows Wet - 7-9-20 #1.TRM,dows wet,2020-07-09,1,


In [12]:
assert len(df_trms.index) == len(trm_filepaths)

In [13]:
df_trms['sample_name'].describe()

count                         484
unique                         69
top       clarion nrf process raw
freq                           30
Name: sample_name, dtype: object

## match the lab report columns

In [14]:
df_lr, lr_filepaths = parse_lab_reports(
    data_dir=data_path,
    skip_paths=skip_paths,
    concurrent=concurrent
)


In [15]:
df_lr.head()

,MVTL Log#,Account #,Customer,State,Date_Sampled,Date_Received,Date_Reported,Sample_ID,Producer,Site Name,...,Cu,Mn,Ca,Mg,Na,pH,EC,filename,sample_name,sample_date
0,20-N6276,1970,FARM NUTRIENTS,IA,,17-Aug-20,19-Aug-20,8,CLARION IA CVF,WEST,...,24.8,361.0,99000.0,3790.0,3240.0,NaN,NaN,Lab Report - West 8 - 8-13-20.csv,west 8,2020-08-13
0,20-N6505,1970,FARM NUTRIENTS,IA,,19-Aug-20,NaN,Process Raw,NaN,Clarion NRF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lab Report - Clarion NRF Process Raw - 8-19-20...,clarion nrf process raw,2020-08-19
0,20-N6482,1970,FARM NUTRIENTS,IA,,19-Aug-20,24-Aug-20,S2,CVF,SOUTH,...,23.1,402.0,112000.0,4270.0,2830.0,NaN,NaN,Lab Report - South 2 - 8-13-20.csv,south 2,2020-08-13
0,20-N4515,1970,FARM NUTRIENTS,IA,,10-Jul-20,13-Jul-20,15,CLARION IA CVF,SOUTH S,...,23.0,346.0,103000.0,4040.0,2670.0,NaN,NaN,Lab Report - South 15 - 7-8-20.csv,south 15,2020-07-08
0,20-N6333,1970,FARM NUTRIENTS,IA,,18-Aug-20,19-Aug-20,16,CVF,NORTH,...,30.3,407.0,108000.0,4220.0,8410.0,NaN,NaN,Lab Report - North 16 - 8-13-20.csv,north 16,2020-08-13


In [16]:
assert len(df_lr.index) == len(lr_filepaths)

In [17]:
unmatched_samples= get_unmatched_sample_ids(df_lr, df_trms)
if len(unmatched_samples) > 0:
    print(f'unable to match sample lab reports named {unmatched_samples}')

In [18]:
lr_to_join = df_lr.set_index(SAMPLE_IDENTIFIER_COLUMNS)[LAB_REPORT_COLUMNS]
df = df_trms.join(lr_to_join, on=SAMPLE_IDENTIFIER_COLUMNS, lsuffix='_trm', rsuffix='_lr')\
                                    .reset_index(drop=True)

In [19]:
df.head()

,extra_info,integration_time,filename_trm,sample_name,sample_date,run_number,process_method,filename_lr,Ammonia-N,N,Moisture,P,K,S
0,File: Users\stell\Desktop\StellarDat\North 15...,40,North 15 - 8-13-20 #1.TRM,north 15,2020-08-13,1,,Lab Report - North 15 - 8-13-20.csv,0.37,3.91,26.0,3.27,2.17,3620.0
1,File: Users\stell\Desktop\StellarDat\South 10...,40,South 10 - 7-8-20 #2.TRM,south 10,2020-07-08,2,,Lab Report - South 10 - 7-8-20.csv,0.32,3.15,32.0,2.39,1.92,2660.0
2,File: Users\stell\Desktop\StellarDat\West 3 -...,30,West 3 - 7-6-20 #3.TRM,west 3,2020-07-06,3,,Lab Report - West 3 - 7-6-20.csv,0.24,2.65,18.7,4.94,3.35,5310.0
3,File: Users\stell\Desktop\StellarDat\West 14 ...,30,West 14 - 8-13-20 #1.TRM,west 14,2020-08-13,1,,Lab Report - West 14 - 8-13-20.csv,0.53,3.96,25.4,3.60,2.27,4240.0
4,File: Users\stell\Desktop\StellarDat\Dows Wet...,35,Dows Wet - 7-9-20 #1.TRM,dows wet,2020-07-09,1,,Lab Report - Dows Wet - 7-9-20.csv,1.12,1.87,76.7,1.13,0.73,1380.0


In [20]:
df['filename_lr'].describe()

count                                    487
unique                                   126
top       Lab Report - South 17 - 7-8-20.csv
freq                                       6
Name: filename_lr, dtype: object

In [21]:
df[df['filename_lr'].isna()]

,extra_info,integration_time,filename_trm,sample_name,sample_date,run_number,process_method,filename_lr,Ammonia-N,N,Moisture,P,K,S


In [22]:
len(df.index)

487

In [23]:
df['filename_trm'].nunique()

484

In [30]:
filename_counts = df['filename_trm'].value_counts()
filename_counts

Hampton Coulter - 6-15-20 #1.TRM    2
Hampton Coulter - 6-15-20 #2.TRM    2
Hampton Coulter - 6-15-20 #3.TRM    2
North 9 - 7-9-20 #2.TRM             1
West 5 - 7-6-20 #1.TRM              1
                                   ..
West 3 (Ground) - 7-6-20 #1.TRM     1
South 15 - 8-13-20 #3.TRM           1
West 12 - 8-13-20 #1.TRM            1
North 1 - 8-13-20 #2.TRM            1
South 4 - 7-8-20 #1.TRM             1
Name: filename_trm, Length: 484, dtype: int64

In [31]:
filename_counts[filename_counts > 1]

Hampton Coulter - 6-15-20 #1.TRM    2
Hampton Coulter - 6-15-20 #2.TRM    2
Hampton Coulter - 6-15-20 #3.TRM    2
Name: filename_trm, dtype: int64

In [28]:
len(df.drop_duplicates(subset=['filename_trm']))

484